## A/B Testing in E-commerce

Author: CHENG CHEN <br/>
LinkedIn: https://www.linkedin.com/in/chengchen-jimmy<br/>
Date: 06/30/2018<br/>

This is a A/B testing tutorial. I will go through a basic A/B testing problem in the e-commerce field. The data comes from BitTiger. Please indicate the source if sharing. 

If you would like to give any suggestions or correct some fields, I could be reached at cheng.chen.2017@marshall.usc.edu. It will be very helpful! Thanks beforehand!

### Business Background & Questions

This data shows user's activities on a online sales website.  
 
An experiment was launched on day 4 with a goal to increase users' click through rate (clicks/views) and total revenue. 
 
Note: first 3 days no experiment, 4th-14th days experiment running 
 
Analyze the data and write a report to answer the following questions: 

1. Is the experiment correct? Please note any problems in your report. 

2. Did the test version help increase CTR and total revenue? Help the team make the right decisions. 

3. There might be learning effect for users to get used to the change. Help the team understand this.

### 1. Import the Dataset

In [68]:
# from pip._internal import main
# main(['install', 'pandasql'])

  Running setup.py bdist_wheel for pandasql: started
  Running setup.py bdist_wheel for pandasql: finished with status 'done'
  Stored in directory: C:\Users\Craig David\AppData\Local\pip\Cache\wheels\53\6c\18\b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


distributed 1.21.8 requires msgpack, which is not installed.


0

In [1]:
## import libraries
import os
import pandas as pd
import pip
import pandasql as ps
import numpy as np
import datetime as datetime


In [2]:
# change work directory
os.chdir('D:\\AB Testing')
# read in the dataset
dat = pd.read_csv('abtest_example_ctr.csv')
# dat.head()
dat.head()

,userid,country,groups,deviceid,device,sellerid,itemid,date,views,clicks,revenue
0,3441.0,US,control,12745,Ios,306,1685,2017-05-09,5,1,0.00000
1,3048.0,GB,control,19244,Android,227,2257,2017-05-09,3,1,0.00000
2,6640.0,CA,treatment,17198,Other,220,2977,2017-05-10,8,2,114.09041
3,4326.0,US,control,5329,Ios,431,1471,2017-05-12,4,0,0.00000
4,6993.0,US,control,15955,Web,399,1812,2017-05-08,4,2,0.00000


### 2. Summary of Statistics

In [3]:
## Summary of statistics
print('Columns are: ', list(dat.columns))
print()
print('Data contains \n', dat.shape[0], ' rows, and ', dat.shape[1], ' columns')
print()
print('Data contains \n', 
      np.unique(dat['country']).size, ' countries \n',
      np.unique(dat['userid']).size, ' userids \n',
      np.unique(dat['deviceid']).size, 'deviceids \n',
      np.unique(dat['sellerid']).size, 'sellerids \n',
      np.unique(dat['itemid']).size, 'itemids \n',
      'Date started from ', min(dat['date']), ' to ', max(dat['date']), ' in total lasts ', \
      (max(pd.to_datetime(dat['date'])) - min(pd.to_datetime(dat['date']))).days + 1, ' days.')

Columns are:  ['userid', 'country', 'groups', 'deviceid', 'device', 'sellerid', 'itemid', 'date', 'views', 'clicks', 'revenue']

Data contains 
 22960  rows, and  11  columns

Data contains 
 4  countries 
 7853  userids 
 7709 deviceids 
 401 sellerids 
 1968 itemids 
 Date started from  2017-05-08  to  2017-05-21  in total lasts  14  days.


In [4]:
# Describe the dataset
np.round(dat.describe(include = 'all'))

,userid,country,groups,deviceid,device,sellerid,itemid,date,views,clicks,revenue
count,22685.0,22960,22960,22960.0,22960,22960.0,22960.0,22960,22960.0,22960.0,22960.0
unique,NaN,4,2,NaN,4,NaN,NaN,14,NaN,NaN,NaN
top,NaN,US,treatment,NaN,Android,NaN,NaN,2017-05-16,NaN,NaN,NaN
freq,NaN,9182,11500,NaN,7003,NaN,NaN,1721,NaN,NaN,NaN
mean,5485.0,NaN,NaN,12566.0,NaN,302.0,1998.0,NaN,6.0,1.0,12.0
std,2584.0,NaN,NaN,4367.0,NaN,116.0,574.0,NaN,3.0,1.0,38.0
min,1000.0,NaN,NaN,5000.0,NaN,100.0,1000.0,NaN,0.0,0.0,0.0
25%,3256.0,NaN,NaN,8758.0,NaN,203.0,1508.0,NaN,4.0,0.0,0.0
50%,5450.0,NaN,NaN,12538.0,NaN,304.0,1994.0,NaN,6.0,1.0,0.0
75%,7717.0,NaN,NaN,16409.0,NaN,402.0,2497.0,NaN,7.0,2.0,0.0


### 3. Data Sanity Check

As this experiment focuses on users' click through rate and total revenue, userid will be the assignment unit of this experiment.

In [5]:
# Check missing values of userid
print('# of misssing userids: ', dat['userid'].isnull().sum())
print('Prop of # missing userid records: ', dat['userid'].isnull().sum()/dat.shape[0]) # less than 1.2%

# of misssing userids:  275
Prop of # missing userid records:  0.011977351916376307


In [6]:
# Check for mixed assignment
# Mixed assignment: same users has been assigned to both control and treatment  

# Here I will be using sql ... for fun
#sql_code = """
#SELECT DISTINCT userid FROM dat WHERE groups = 'treatment'
#INTERSECT
#SELECT DISTINCT userid FROM dat WHERE groups = 'control';
#"""
#pysqldf = lambda q: ps.sqldf(q, globals())
#mixAssign = ps.sqldf(sql_code)

mixAssign = set(dat.loc[dat.groups == 'control', 'userid']) & set(dat.loc[dat.groups == 'treatment', 'userid'])
len(mixAssign)

43

In [7]:
# Check for multiple-user per device
mulUser = dat.groupby('deviceid')['userid'].nunique()
mulUser = set(mulUser[mulUser>1].index)
len(mulUser)

136

In [8]:
# Check for multiple-device per user
mulDev = dat.groupby('userid')['deviceid'].nunique()
mulDev = set(mulDev[mulDev>1].index)
len(mulDev)

175

In [9]:
# Check if NA/mixed/multiple-device/multiple-user is random
# Create dummy, if any problem 1, else 0.
dat['Deficit'] = dat.apply(lambda x: int((x['userid'] in mixAssign|mulDev) | np.isnan(x['userid']) | (x['deviceid'] in mulUser)), axis = 1)
dat['Deficit'].sum()

1823

For simplicity, we run a logistic regression model and check the p-value of each variables to see if the missing is associate with specific country/groups/device/date/views/clicks/revenue.

But in the first place, to use sklearn, we need to 'dummy' those variables


In [22]:
# Create dummy variables
datCheck = dat.copy()
datCheck = pd.get_dummies(datCheck[['country', 'groups', 'device', 'date', 'views', 'clicks', 'revenue', 'Deficit']])
datCheck.drop(columns = ['country_CA', 'groups_control', 'date_2017-05-08', 'device_Android'], inplace = True)
datCheck.head()

,views,clicks,revenue,Deficit,country_CN,country_GB,country_US,groups_treatment,device_Ios,device_Other,...,date_2017-05-12,date_2017-05-13,date_2017-05-14,date_2017-05-15,date_2017-05-16,date_2017-05-17,date_2017-05-18,date_2017-05-19,date_2017-05-20,date_2017-05-21
0,5,1,0.00000,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1,0.00000,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,2,114.09041,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0.00000,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,4,2,0.00000,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Run a simple logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0, C=1e6) # lower C means higher penalty, use convention in SVM
lr.fit(datCheck.loc[:,datCheck.columns!='Deficit'], datCheck['Deficit'])

LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [30]:
# Check the attributes
print('intercept: ', lr.intercept_)
dict(zip(datCheck.columns[datCheck.columns!='Deficit'], lr.coef_[0]))

# Note:
# sklearn doesn't provide built-in P-value and other statistical calculation 

intercept:  [-2.44275683]


{'views': 0.003676205223950213,
 'clicks': -0.032457761797823496,
 'revenue': 0.0002996713985630129,
 'country_CN': -0.12864775250729915,
 'country_GB': -0.15981963077922479,
 'country_US': -0.09787344286589661,
 'groups_treatment': -0.028145905795962302,
 'device_Ios': 0.13190803322761677,
 'device_Other': 0.18063270969294815,
 'device_Web': 0.11274798162673037,
 'date_2017-05-09': -0.09359675762579389,
 'date_2017-05-10': -0.027372523582008923,
 'date_2017-05-11': 0.028565325749638566,
 'date_2017-05-12': 0.09078756077920669,
 'date_2017-05-13': 0.0347781267873893,
 'date_2017-05-14': 0.2117121449867692,
 'date_2017-05-15': -0.0975895380608316,
 'date_2017-05-16': -0.006356662512770055,
 'date_2017-05-17': 0.03814663794821731,
 'date_2017-05-18': -0.024250236173639826,
 'date_2017-05-19': 0.049119346515590384,
 'date_2017-05-20': 0.03767493806071268,
 'date_2017-05-21': -0.03397509740435897}

In [35]:
# Alternative logistic regression packages/api
# use statsmodels.api to get the P-value
import statsmodels.api as sm
datCheck['intercept'] = 1 # Note, if we don't have the intercept, statsmodels will by default has not intercept
model = sm.Logit(datCheck['Deficit'], datCheck.loc[:,datCheck.columns!='Deficit'])
result = model.fit()
result.summary()
# if we have P-value, we should be able to know that country_GB and device_Other are significant in the logistic regression.
# we can dive deep to see what happens there.

Optimization terminated successfully.
         Current function value: 0.276770
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Deficit   No. Observations:                22960
Model:                          Logit   Df Residuals:                    22936
Method:                           MLE   Df Model:                           23
Date:                Sat, 30 Jun 2018   Pseudo R-squ.:                0.001908
Time:                        22:41:59   Log-Likelihood:                -6354.6
converged:                       True   LL-Null:                       -6366.8
                                        LLR p-value:                    0.3874
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
views                0.0037      0.011      0.326      0.745      -0.019       0.026
clicks              -0.0325      0.027     -1.225      0.221      -0.084       0.019
revenue              0.0003      0.001      0.456      0.648      -0.001       0.002
country_CN          -0.1284      0.076     -1.679      0.093      -0.278       0.021
country_GB          -0.1596      0.077     -2.079      0.038      -0.310      -0.009
country_US          -0.0977      0.065     -1.497      0.134      -0.226       0.030
groups_treatment    -0.0280      0.049     -0.574      0.566      -0.124       0.068
device_Ios           0.1320      0.071      1.859      0.063      -0.007       0.271
device_Other         0.1807      0.070      2.599      0.009       0.044       0.317
device_Web           0.1129      0.065      1.748      0.080      -0.014       0.239
date_2017-05-09     -0.0935      0.134     -0.696      0.487      -0.357       0.170
date_2017-05-10     -0.0273      0.133     -0.206      0.837      -0.288       0.233
date_2017-05-11      0.0286      0.131      0.218      0.828      -0.229       0.286
date_2017-05-12      0.0908      0.130      0.701      0.483      -0.163       0.345
date_2017-05-13      0.0348      0.130      0.267      0.790      -0.221       0.290
date_2017-05-14      0.2119      0.127      1.662      0.096      -0.038       0.462
date_2017-05-15     -0.0975      0.134     -0.729      0.466      -0.360       0.165
date_2017-05-16     -0.0064      0.131     -0.049      0.961      -0.263       0.250
date_2017-05-17      0.0381      0.131      0.292      0.770      -0.218       0.294
date_2017-05-18     -0.0242      0.133     -0.182      0.856      -0.286       0.237
date_2017-05-19      0.0492      0.131      0.375      0.708      -0.208       0.306
date_2017-05-20      0.0377      0.131      0.288      0.774      -0.219       0.295
date_2017-05-21     -0.0339      0.134     -0.253      0.800      -0.297       0.229
intercept           -2.4432      0.130    -18.832      0.000      -2.697      -2.189
====================================================================================
"""

*Very Important*: we can see the coefficient lines up with the previous one.

Let's compare three ways of logistic regression calculation:
* R: no need to create dummy and has statistics. use code: "m = glm(y ~ x1 + x2 + ... + x3, data, family = 'binomial') \n summary(m)"
* sklearn: needs to create dummy variables; no p-value & other statistics.
* statsmodels.api: needs to create dummy variables and need to specify 'intercept', otherwise intercept is not there by default.

In [43]:
# Example: Dive deep into device_other (similar analysis could be applied on Country_GB)
# since the device_other has a significant effect, let dive deep into this
dat['nullUser'] = dat['userid'].isnull()*1
dat['mixAssign'] = dat['userid'].apply(lambda x: int(x in mixAssign))
dat['mulUser'] = dat['deviceid'].apply(lambda x: int(x in mulUser))
dat['mulDev'] = dat['userid'].apply(lambda x: int(x in mulDev))
# sum(np.sum(dat[['nullUser','mixAssign','mulUser','mulDev']],axis=1)>=1) # 1823 Checked!
print(dat.groupby('device')['nullUser'].mean())
print(dat.groupby('device')['mixAssign'].mean())
print(dat.groupby('device')['mulUser'].mean()) # higher multiple user percent - is it expected? bug? talk to engineers.
print(dat.groupby('device')['mulDev'].mean())

device
Android    0.013708
Ios        0.008946
Other      0.009752
Web        0.013820
Name: nullUser, dtype: float64
device
Android    0.004998
Ios        0.004582
Other      0.007632
Web        0.008863
Name: mixAssign, dtype: float64
device
Android    0.029702
Ios        0.037530
Other      0.043884
Web        0.029743
Name: mulUser, dtype: float64
device
Android    0.038983
Ios        0.050185
Other      0.045792
Web        0.044765
Name: mulDev, dtype: float64
